In [1]:
if(!require('dunn.test')) {
  install.packages('dunn.test')
  library('dunn.test')
}

Lade nötiges Paket: dunn.test

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“es gibt kein Paket namens ‘dunn.test’”



Die heruntergeladenen Binärpakete sind in 
	/var/folders/lb/nxxqtlbj7yd1c5frnp5_vxk40000gn/T//RtmpscFBWO/downloaded_packages


In [13]:
## Example cribbed and modified from the kruskal.test documentation
## Hollander & Wolfe (1973), 116.
## Mucociliary efficiency from the rate of removal of dust in normal
##  subjects, subjects with obstructive airway disease, and subjects
##  with asbestosis.  
veh <- c(6.22, 6.27, 5.2, 6.24, 6.49) # vehicle
har <- c(6.52, 6.62, 6.42, 6.59, 6.37, 6.72)      # harmine
dmt <- c(6.37, 6.32, 6.19, 6.35, 6.25, 6.49) # DMT
hardmt <- c(6.75, 6.4, 6.2, 6.16, 6.38, 6.32) # har + DMT
dunn.test(x=list(veh,har,dmt,hardmt))

x <- c(veh,har,dmt,hardmt)
g <- factor(rep(1:4, c(5, 6, 6, 6)),
            labels = c("Vehicle",
                       "Harmine", 'DMT', 'har + DMT'))
dunn.test(x, g, method='sidak', altp=TRUE, list=TRUE, rmc=TRUE)


  Kruskal-Wallis rank sum test

data: x and group
Kruskal-Wallis chi-squared = 8.1362, df = 3, p-value = 0.04


                           Comparison of x by group                            
                                (No adjustment)                                
Col Mean-|
Row Mean |          1          2          3
---------+---------------------------------
       2 |  -2.660097
         |    0.0039*
         |
       3 |  -0.629488   2.129720
         |     0.2645    0.0166*
         |
       4 |  -0.913773   1.831559  -0.298160
         |     0.1804     0.0335     0.3828

alpha = 0.05
Reject Ho if p <= alpha/2
  Kruskal-Wallis rank sum test

data: x and g
Kruskal-Wallis chi-squared = 8.1362, df = 3, p-value = 0.04


                             Comparison of x by g                              
                                    (Šidák)                                    
Row Mean-|
Col Mean |        DMT   har + DM    Harmine
---------+---------------------------------
ha

In [8]:
dunn.test

function (x = NA, g = NA, method = p.adjustment.methods, kw = TRUE, 
    label = TRUE, wrap = FALSE, table = TRUE, list = FALSE, rmc = FALSE, 
    alpha = 0.05, altp = FALSE) 
{
    kwallis.test <- function(x = NA, g = NA, pass = 0) {
        tiedranks <- function(ranks) {
            ranks <- sort(ranks)
            ties <- c()
            for (i in 2:length(ranks)) {
                if (ranks[i - 1] == ranks[i]) {
                  if (length(ties) > 0) {
                    if (ranks[i - 1] != tail(ties, n = 1)) {
                      ties <- c(ties, ranks[i - 1])
                    }
                  }
                  else {
                    ties <- c(ranks[i - 1])
                  }
                }
            }
            return(ties)
        }
        if (is.list(x)) {
            N <- 0
            for (i in 1:length(x)) {
                N <- N + length(x[[i]])
            }
            Data <- matrix(NA, N, 4)
            for (i in 1:N) {
                Data[i, 1] <- i
            }
            obs <- c()
            group <- c()
            for (i in 1:length(x)) {
                obs <- c(obs, x[[i]])
                group <- c(group, rep(i, length(x[[i]])))
            }
            Data[, 2] <- obs
            if (length(g) > 1) {
                Data[, 3] <- g
            }
            else {
                Data[, 3] <- group
            }
            Data[, 4] <- rank(Data[, 2], ties.method = "average", 
                na.last = NA)
        }
        if (!is.list(x)) {
            N <- length(x)
            Data <- matrix(NA, length(x), 4)
            Data[, 1] <- 1:length(x)
            Data[, 2] <- x
            Data[, 3] <- g
            Data[, 4] <- rank(Data[, 2], ties.method = "average", 
                na.last = NA)
        }
        k <- length(unique(Data[, 3]))
        ranks <- Data[, 4]
        ranks <- ranks[!is.na(ranks)]
        ties <- tiedranks(ranks)
        r <- length(ties)
        tiesadj <- 0
        if (r > 0) {
            for (s in 1:r) {
                tau <- sum(ranks == ties[s])
                tiesadj <- tiesadj + (tau^{
                  3
                } - tau)
            }
        }
        tiesadj <- 1 - (tiesadj/((N^3) - N))
        ranksum <- 0
        for (i in 1:k) {
            ranksum <- ranksum + ((sum(Data[, 4][Data[, 3] == 
                i]))^2)/(sum(Data[, 3] == i))
        }
        H <- ((12/(N * (N + 1))) * ranksum - 3 * (N + 1))/tiesadj
        df <- k - 1
        p <- pchisq(H, k - 1, lower.tail = FALSE)
        output <- paste("Kruskal-Wallis chi-squared = ", round(H, 
            digits = 4), ", df = ", df, ", p-value = ", round(p, 
            digits = 2), "\n\n", sep = "")
        invisible(list(output = output, H = H, df = df, p = p, 
            N = N, Data = Data, data.name = data))
    }
    all.integers <- function(x) {
        for (i in length(x)) {
            if (is.na(x[i]) | is.list(x[i]) | length(x[i]) > 
                1 | !is.numeric(x[i])) {
                return(FALSE)
            }
            if (x[i]%%1 != 0) {
                return(FALSE)
            }
        }
        return(TRUE)
    }
    tiedranks <- function(ranks) {
        ranks <- sort(ranks)
        ties <- c()
        for (i in 2:length(ranks)) {
            if (ranks[i - 1] == ranks[i]) {
                if (length(ties) > 0) {
                  if (ranks[i - 1] != tail(ties, n = 1)) {
                    ties <- c(ties, ranks[i - 1])
                  }
                }
                else {
                  ties <- c(ranks[i - 1])
                }
            }
        }
        return(ties)
    }
    tpad <- function(n = 1, pad = " ") {
        if (n == 0) {
            return("")
        }
        return(paste0(rep(x = pad, times = n), collapse = ""))
    }
    zformat <- function(z) {
        if (z < 0) {
            sign <- "-"
        }
        else {
            sign <- " "
        }
        leftspaces <- max(2, floor(log10(abs(z))) + 2)
      